In [38]:
!pip install -qqr requirements.txt
# !pip install -U selenium unstructured

: 

In [6]:
from langchain.document_loaders import SeleniumURLLoader

In [7]:
urls=[
    "https://www.moneycontrol.com/india/stockpricequote/ironsteel/tatasteel/TIS",
    "https://www.moneycontrol.com/india/stockpricequote/auto-lcvshcvs/tatamotors/TM03",
]

In [8]:
loader = SeleniumURLLoader(urls=urls)

In [9]:
data = loader.load()

In [26]:
text = data[0]
text.page_content

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nTrending Stocks\n\nTCS\xa0INE467B01029, TCS, 532540\n\nBharti Hexacom\xa0INE343G01021, BHARTIHEXA, 544162\n\nVodafone Idea\xa0INE669E01016, IDEA, 532822\n\nYes Bank\xa0INE528G01035, YESBANK, 532648\n\nJio Financial\xa0INE758E01017, JIOFIN, 543940\n\n\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, Login\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tLog-in\n\t\t\t\t\t\t\t\t\t\t\tor \n\t\t\t\t\t\t\t\t\t\t\tSign-Up\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\tMy Account\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tMy Profile \n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tMy Portfolio\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],
    chunk_size = 200,
    chunk_overlap = 10
)

chunks = r_splitter.split_text(data[0].page_content)

In [34]:
print(f"{len(chunks)}_")
for chunk in chunks:
    print(len(chunk))

713_
173
194
21
156
167
167
12
13
54
136
105
148
153
84
92
103
112
19
109
148
137
98
156
88
5
7
16
16
3
99
83
137
80
146
145
11
3
82
154
83
150
141
81
77
154
82
143
125
141
78
154
137
11
49
136
147
101
161
113
97
90
104
176
140
140
151
138
153
13
110
171
130
82
104
149
110
107
139
32
5
49
132
34
139
154
143
91
133
86
128
87
178
82
11
6
79
170
109
153
126
84
174
82
144
169
103
73
121
66
63
145
10
192
194
196
192
72
172
73
65
74
85
85
75
59
158
83
66
101
74
161
150
63
66
76
124
164
79
81
88
13
153
11
26
154
93
177
142
147
122
198
195
107
196
19
197
175
178
96
198
104
195
70
198
104
110
184
1
161
151
151
153
156
151
153
156
151
153
187
177
136
116
131
178
139
152
198
193
195
193
52
197
154
48
149
156
121
136
146
138
147
117
105
154
164
127
4
29
1
28
2
25
81
45
33
145
144
194
102
141
151
152
39
141
141
190
100
137
147
87
196
23
162
11
195
189
182
190
158
132
194
193
182
159
145
55
12
190
169
111
149
140
146
41
143
154
171
33
154
171
33
154
171
33
144
80
121
193
70
145
55
12
190
160
163
163

we will use Faiss (Facebook ai similar search)

In [6]:
import os
import streamlit as st
import pickle
import time
from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore
import tempfile

# st.title("Website Research ...")
# st.sidebar.title("URLs to articels")


main_placefolder = st.empty()
button = st.empty()
ans = st.empty()


urls = ["https://en.wikipedia.org/wiki/Larry_Sanger"]

# for i in range (1):
#     url = st.sidebar.text_input(f"URL {i+1}")
#     urls.append(url)
#     main_placefolder.write(url)
# processURL_clicked = st.sidebar.button("Process URL")


# for url in urls:
#     st.sidebar.write(url)

llm = Ollama(model="mistral")

# if processURL_clicked:
    # with st.status("Loading Data ...", expanded=False):
        #load data
loader = WebBaseLoader(urls)
data = loader.load()

#split data
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(data)

    #save embeddings
embeddings = OllamaEmbeddings(model="mistral")
from pathlib import Path
persist_path = Path("./vectore_store.pkl")
#persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
# with st.status("Making embeddings ...", expanded=False):
#        vector_store = SKLearnVectorStore.from_documents(
#            documents=docs,
#            embedding=embeddings,
#            persist_path=persist_path,  # persist_path and serializer are optional
#            serializer="parquet",
#        )
#        vector_store.persist()
vector_store = SKLearnVectorStore(embedding=embeddings,
    persist_path=persist_path,  # persist_path and serializer are optional
    serializer="parquet", )


    # main_placefolder.text("done")
    # time.sleep(1)

question = "why are there more clouds in the afternoon?"

from langchain import hub   

    # Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")


from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
        llm, retriever=vector_store.as_retriever(), chain_type_kwargs={"prompt": prompt}
    )

result = qa_chain({"query": question})
# main_placefolder.write(result["result"])
    #result = qa_chain({"query": question})
    #ans.write(result['result'])
result['result']


{'query': 'why are there more clouds in the afternoon?',
 'result': ' Thunderstorms contribute to cloud formation in the afternoon by providing moisture and cooling the Earth, leading to increased humidity and condensation. (Use two sentences)'}